We will build a regression model for laptop price dataset. The dataset consists of  feature, which are independent and charge as a dependent feature. We will predict laptops costs.

## Import Library and Dataset
Now we will import couple of python library required for our analysis and import dataset 

Import Library

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, ConfusionMatrixDisplay, classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from io import StringIO
from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns


plt.rcParams['font.size'] =10


ModuleNotFoundError: No module named 'matplotlib'

Import Dataset

In [ ]:
# Import dataset
df = pd.read_csv('laptop_price.csv')
print('\nNumber of rows and columns in the data set: ',df.shape)
print('')

#Lets look into top few rows and columns in the dataset
df.head()

Data Shape

In [ ]:
# Data Shape
print("Data Shape:", df.shape)


Data Types

In [ ]:
# Data Types
print("\nData Types:\n", df.dtypes)

In [ ]:
def preprocess(df):
    # Create a copy to avoid modifying the original DataFrame
    df = df.copy()
    
    # 1. ScreenResolution processing
    df['ScreenResolution'] = df['ScreenResolution'].astype(str).fillna('Unknown')
    
    # Extract resolution and screen type
    resolution_split = df['ScreenResolution'].str.split(' ', expand=False)
    df['Resolution'] = resolution_split.str[-1]
    df['Screen'] = resolution_split.apply(lambda x: ' '.join(x[:-1]) if len(x) > 1 else 'Standard')
    
    # Extract width and height from resolution (e.g., "1920x1080")
    resolution_parts = df['Resolution'].str.split('x', expand=True)
    df['ScreenW'] = resolution_parts[0].replace('', '0').fillna('0').astype('float')
    df['ScreenH'] = resolution_parts[1].replace('', '0').fillna('0').astype('float')   
    # Screen features extraction
    df['Touchscreen'] = df['Screen'].str.contains('Touchscreen').map({True: 'Yes', False: 'No'})
    df['IPSpanel'] = df['Screen'].str.contains('IPS Panel').map({True: 'Yes', False: 'No'})
    df['RetinaDisplay'] = df['Screen'].str.contains('Retina Display').map({True: 'Yes', False: 'No'})
    
    # Clean up screen description
    df['Screen'] = df['Screen'].str.replace('Touchscreen|IPS Panel|Retina Display|/', '', regex=True)
    df['Screen'] = df['Screen'].str.strip()
    df['Screen'] = np.where(df['Screen'].str.len() > 0, df['Screen'], 'Standard')
    df.drop(inplace=True, columns=['ScreenResolution'])

    # 2. CPU processing - initialize with string dtype
    df['CPU_brand'] = ''
    df['CPU_model_and_generation'] = ''
    df['CPU_base_clock_speed'] = ''
    
    for index, value in df['Cpu'].items():
        if pd.isna(value):
            continue
            
        parts = str(value).split()
        if not parts:
            continue
            
        # Brand (first word)
        df.at[index, 'CPU_brand'] = parts[0]
        
        # Model and generation (middle words)
        if len(parts) > 1:
            model_parts = parts[1:-1] if len(parts) > 2 else [parts[1]]
            df.at[index, 'CPU_model_and_generation'] = ' '.join(model_parts)
        
        # Clock speed (last word)
        if len(parts) > 1:
            df.at[index, 'CPU_base_clock_speed'] = parts[-1]


    # 3. Memory/Storage processing
    # Ensure Memory is string type and handle NaN values
    df.drop(inplace=True, columns=['Cpu'])

    df['Memory'] = df['Memory'].astype(str)
    
    # Split memory components
    df['Memory'] = df['Memory'].str.split('+').apply(lambda x: [part.strip() for part in x])
    
    # Primary storage
    df['PrimaryStorage'] = df['Memory'].apply(lambda x: x[0] if len(x) > 0 else '0')
    df['PrimaryStorageType'] = df['PrimaryStorage'].str.extract(r'(\D+)$')[0].str.strip()
    df['PrimaryStorage'] = df['PrimaryStorage'].str.extract(r'^(\d+)')[0].fillna(np.nan)
    
    # Secondary storage
    df['SecondaryStorage'] = df['Memory'].apply(
        lambda x: x[1] if len(x) > 1 else '0'
    )
    secondary_split = df['SecondaryStorage'].str.split(n=1, expand=True)
    df['SecondaryStorageType'] = secondary_split[1].fillna('No')
    df['SecondaryStorage'] = secondary_split[0].fillna('0')
    df['PrimaryStorage'] = df['PrimaryStorage'].apply(convert_to_gb).astype('int64')
    df['SecondaryStorage'] = df['SecondaryStorage'].apply(convert_to_gb).astype('int64')
    df.drop(columns=['Memory'], inplace=True)

    # 4. GPU processing
    df['Gpu'] = df['Gpu'].astype(str)
    gpu_split = df['Gpu'].str.split(n=1, expand=True)
    df['GPU_company'] = gpu_split[0]
    df['GPU_model'] = gpu_split[1].fillna('Unknown')
    df.drop(columns=['Gpu'], inplace=True)

    return df


In [ ]:
def convert_to_gb(value):
    # Convert storage values to gigabytes.
    if pd.isna(value):  # Handle NaN/None values
        return 0.0
        
    value = str(value).strip().upper()  # Ensure string and standardize case
    
    try:
        if 'TB' in value:
            return float(value.replace('TB', '')) * 1024
        elif 'GB' in value:
            return float(value.replace('GB', ''))
        elif 'MB' in value:  # Handle megabytes if needed
            return float(value.replace('MB', '')) / 1024
        else:  # Assume value is already in GB or a raw number
            return float(value)
    except (ValueError, TypeError):
        return 0.0

In [ ]:
df=preprocess(df)
df.head()

In [ ]:
print("\nSummary Statistics:\n", df.describe())


Check for missing values

In [ ]:
print(df.head())

In [ ]:
df.drop(columns=['laptop_ID','Resolution'], inplace=True)


In [ ]:
    df["Weight"] = df["Weight"].str.replace("kg","").astype(float)
    df["Ram"] = df["Ram"].str.replace("GB", "").astype(float)
    df["CPU_base_clock_speed"] = df["CPU_base_clock_speed"].str.replace("GHz","").astype(float)

In [ ]:
df.to_csv("processed_data.csv", index=False)


### Data Visualization

1- Visualizing feature

1. Price Distribution by Company

In [ ]:
plt.figure(figsize=(14, 6))
sns.boxplot(x='Company', y='Price_euros', data=df)
plt.title('Price Distribution by Company', fontsize=16)
plt.xlabel('Company', fontsize=14)
plt.ylabel('Price (Euros)', fontsize=14)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

2. Most Common Laptop Types

In [ ]:
plt.figure(figsize=(10, 6))
type_counts = df['TypeName'].value_counts()
sns.barplot(x=type_counts.index, y=type_counts.values)
plt.title('Most Common Laptop Types', fontsize=16)
plt.xlabel('Type', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


3. Price vs. Weight

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Weight', y='Price_euros', hue='Company', data=df, alpha=0.7)
plt.title('Price vs. Weight by Company', fontsize=16)
plt.xlabel('Weight (kg)', fontsize=14)
plt.ylabel('Price (Euros)', fontsize=14)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


4. RAM Distribution

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='Ram', data=df, order=df['Ram'].value_counts().index)
plt.title('RAM Distribution', fontsize=16)
plt.xlabel('RAM (GB)', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.tight_layout()
plt.show()

5. Operating System Distribution

In [ ]:
plt.figure(figsize=(10, 6))
os_counts = df['OpSys'].value_counts()
sns.barplot(x=os_counts.index, y=os_counts.values)
plt.title('Operating System Distribution', fontsize=16)
plt.xlabel('Operating System', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

6. Screen Size Distribution

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='Inches', data=df)
plt.title('Screen Size Distribution', fontsize=16)
plt.xlabel('Screen Size (Inches)', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.tight_layout()
plt.show()


7. CPU Brand Distribution

In [ ]:
plt.figure(figsize=(10, 6))
cpu_counts = df['CPU_brand'].value_counts()
sns.barplot(x=cpu_counts.index, y=cpu_counts.values)
plt.title('CPU Brand Distribution', fontsize=16)
plt.xlabel('CPU Brand', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.tight_layout()
plt.show()

8. GPU Company Distribution

In [ ]:
plt.figure(figsize=(10, 6))
gpu_counts = df['GPU_company'].value_counts()
sns.barplot(x=gpu_counts.index, y=gpu_counts.values)
plt.title('GPU Company Distribution', fontsize=16)
plt.xlabel('GPU Company', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.tight_layout()
plt.show()

9. Storage Type Distribution

In [ ]:
plt.figure(figsize=(12, 6))
primary_storage = df['PrimaryStorageType'].value_counts()
secondary_storage = df['SecondaryStorageType'].value_counts()

plt.subplot(1, 2, 1)
sns.barplot(x=primary_storage.index, y=primary_storage.values)
plt.title('Primary Storage Type', fontsize=14)
plt.xlabel('Storage Type', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
sns.barplot(x=secondary_storage.index, y=secondary_storage.values)
plt.title('Secondary Storage Type', fontsize=14)
plt.xlabel('Storage Type', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

10.Correlation matrix Heatmap

In [ ]:
# Select numerical columns for correlation
numerical_cols = ['Inches', 'Ram', 'Weight', 'Price_euros', 'ScreenW', 'ScreenH']
plt.figure(figsize=(10, 8))
corr = df[numerical_cols].corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Heatmap', fontsize=16)
plt.tight_layout()
plt.show()


11. Price Distribution by CPU Brand

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='CPU_brand', y='Price_euros', data=df)
plt.title('Price Distribution by CPU Brand', fontsize=16)
plt.xlabel('CPU Brand', fontsize=14)
plt.ylabel('Price (Euros)', fontsize=14)
plt.tight_layout()
plt.show()

12. Average Price by Company and Type

In [ ]:

plt.figure(figsize=(14, 8))
avg_price = df.groupby(['Company', 'TypeName'])['Price_euros'].mean().unstack()
sns.heatmap(avg_price, annot=True, fmt=".0f", cmap="YlGnBu")
plt.title('Average Price by Company and Type', fontsize=16)
plt.xlabel('Type', fontsize=14)
plt.ylabel('Company', fontsize=14)
plt.tight_layout()
plt.show()


13. Touchscreen vs Non-Touchscreen Price Comparison

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(x='Touchscreen', y='Price_euros', data=df)
plt.title('Price Comparison: Touchscreen vs Non-Touchscreen', fontsize=16)
plt.xlabel('Touchscreen', fontsize=14)
plt.ylabel('Price (Euros)', fontsize=14)
plt.tight_layout()
plt.show()


14. Screen Resolution Analysis

In [ ]:
# Create a new column for resolution
df['Resolution'] = df['ScreenW'].astype(str) + 'x' + df['ScreenH'].astype(str)
top_resolutions = df['Resolution'].value_counts().head(10)

plt.figure(figsize=(12, 6))
sns.barplot(x=top_resolutions.index, y=top_resolutions.values)
plt.title('Top 10 Screen Resolutions', fontsize=16)
plt.xlabel('Resolution', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Data cleaning

Clean the data by addressing outliers and converting categorical features to numerical representations.


In [ ]:
# Create a copy of the original DataFrame
# Apply quantile-based outlier removal

for col in ['Price_euros','Ram','Weight', 'Inches']:
    p_lower = 0.01  # 1st percentile
    p_upper = 0.97  # 97th percentile
    lower_bound = df[col].quantile(p_lower)
    upper_bound = df[col].quantile(p_upper)
    
    # Clip values beyond the range
    df[col] = np.clip(df[col], lower_bound, upper_bound)

# Boxplots after outlier treatment
modified_cols = ['Price_euros','Ram', 'Weight', 'Inches']
for col in modified_cols:
    plt.figure(figsize=(8, 6))
    sns.boxplot(y=df[col])  # Show distribution after outlier removal
    plt.title(f'Boxplot of {col} after outliers treatment', fontsize=16)
    plt.xlabel(col, fontsize=14)
    plt.ylabel("Value", fontsize=14)

plt.show()

droping columns that are too detailed 

In [ ]:
df_model=df.drop(columns=['Product','CPU_model_and_generation','GPU_model','Resolution'])

In [ ]:
df_model.to_csv("preprosseced.csv",)
print(df_model.dtypes)

Encoding categorical features

In [ ]:
label_encoders={}
for column in df_model.select_dtypes(include='object').columns:
    le=LabelEncoder()
    df_model[column]=le.fit_transform(df_model[column].astype(str))
    label_encoders[column]=le
    

In [ ]:
df_model.head()

In [ ]:
numerical_cols = df.select_dtypes(include=np.number)

correlation_matrix = numerical_cols.corr()
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Numerical Features')
plt.show()

In [ ]:
df_model.to_csv("encoded_dataset.csv")
print(df_model.dtypes)

split data 

In [ ]:
X = df_model.drop(columns=['Price_euros'])
y = df_model['Price_euros']
feature_columns = X.columns.tolist()

train split 

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size= 0.2,random_state=42)


model1: random forest regressor

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def evaluate_regressors(X_train, X_test, y_train, y_test, X_cols):
    models = {
        "Linear Regression": LinearRegression(),
        "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
        "Decision Tree": DecisionTreeRegressor(random_state=42),
        "Gradient Boosting": GradientBoostingRegressor(random_state=42),
        "KNN Regressor": KNeighborsRegressor(n_neighbors=5)
    }

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        r2 = r2_score(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)

        print(f"\n=== {name} ===")
        print(f"R²:  {r2:.4f}")
        print(f"MSE: {mse:.2f}")
        print(f"MAE: {mae:.2f}")

        # Residual Plot
        plt.figure(figsize=(6, 4))
        residuals = y_test - y_pred
        sns.scatterplot(x=y_pred, y=residuals)
        plt.axhline(y=0, color='r', linestyle='--')
        plt.title(f"{name} - Residual Plot")
        plt.xlabel("Predicted")
        plt.ylabel("Residuals")
        plt.tight_layout()
        plt.show()

        # Predicted vs Actual
        plt.figure(figsize=(6, 4))
        sns.scatterplot(x=y_test, y=y_pred)
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
        plt.title(f"{name} - Predicted vs Actual")
        plt.xlabel("Actual")
        plt.ylabel("Predicted")
        plt.tight_layout()
        plt.show()

        # Feature Importance (Tree Models Only)
        if hasattr(model, 'feature_importances_'):
            importances = model.feature_importances_
            indices = np.argsort(importances)[::-1]
            names = [X_cols[i] for i in indices]

            plt.figure(figsize=(10, 4))
            plt.title(f"{name} - Feature Importance")
            plt.bar(range(len(importances)), importances[indices])
            plt.xticks(range(len(importances)), names, rotation=90)
            plt.tight_layout()
            plt.show()

        # Decision Tree Plot (Optional for Decision Tree only)
        if name == "Decision Tree":
            plt.figure(figsize=(16, 6))
            plot_tree(model, feature_names=X_cols, filled=True, max_depth=3, fontsize=6)
            plt.title("Decision Tree (max_depth=4)")
            plt.tight_layout()
            plt.show()


In [ ]:
evaluate_regressors(X_train, X_test, y_train, y_test, X.columns.tolist())
